# 필요 패키지 import

In [1]:
!pip install KoBERT-master.zip

Looking in indexes: http://10.188.207.107:9091/repository/pypi/simple/
Processing ./KoBERT-master.zip
  Preparing metadata (setup.py) ... done
  Created wheel for kobert: filename=kobert-0.2.3-py3-none-any.whl size=15692 sha256=b65af6d9674dbd8fb5d9f1b8c6c4490a6858dc9346aeec2c1d8fd1bb18ee8a8d
  Stored in directory: /home/jovyan/.cache/pip/wheels/87/20/d6/157ce8d441cac97f8d59ca5e897e432efb0f815f4fbe401660
Successfully built kobert
  Attempting uninstall: kobert
    Found existing installation: kobert 0.2.3
    Uninstalling kobert-0.2.3:
      Successfully uninstalled kobert-0.2.3

[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: pip install --upgrade pip


In [2]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm

In [3]:
from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model

In [4]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

2022-11-23 00:25:42.382966: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [5]:
from transformers import BertModel

bertmodel=BertModel.from_pretrained('./kobert_from_pretrained', return_dict=False)

In [6]:
#device = torch.device("cpu")
#gpu 환경에서 실행
device = torch.device("cuda:0")

In [7]:
bertmodel.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(8002, 768, padding_idx=1)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )

In [8]:
bertmodel.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(8002, 768, padding_idx=1)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )

In [9]:
vocab = nlp.vocab.BERTVocab.from_sentencepiece('./kobert_news_wiki_ko_cased-1087f8699e.spiece', padding_token = "[PAD]")

# 데이터셋 불러오기

In [10]:
#train test 셋으로 나눈 데이터를 import 하여 dataset_train, dataset_test에 각각 지정
dataset_train = nlp.data.TSVDataset('tsv_dataset_train.txt', field_indices=[1,2] ,num_discard_samples =1)
dataset_test = nlp.data.TSVDataset('tsv_dataset_test.txt', field_indices=[1,2], num_discard_samples =1)

In [11]:
from gluonnlp.data import SentencepieceTokenizer

#tok_path = './kobert_news_wiki_ko_cased-1087f8699e.spiece'
#tokenizer = SentencepieceTokenizer(tok_path)

tokenizer = './kobert_news_wiki_ko_cased-1087f8699e.spiece'

In [12]:
#tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower = False)

In [13]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [14]:
#세부 파라미터 지정
max_len = 200
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [15]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [16]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [17]:
#BERT 모델 불러오기
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        else:
            out = pooler
        return self.classifier(out)

In [18]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [19]:
#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}]

In [20]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
#crossentropy 함수 옵션 설정
loss_fn = nn.CrossEntropyLoss()

In [21]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [22]:
#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

# 모델 적용

In [23]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

  0%|          | 0/118 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.7199442386627197 train acc 0.609375
epoch 1 train acc 0.6733597874409558


  0%|          | 0/40 [00:00<?, ?it/s]

epoch 1 test acc 0.7588169642857143


  0%|          | 0/118 [00:00<?, ?it/s]

...

epoch 9 test acc 0.8936941964285714


  0%|          | 0/118 [00:00<?, ?it/s]

epoch 10 batch id 1 loss 0.10893259197473526 train acc 0.96875
epoch 10 train acc 0.9809126667129758


  0%|          | 0/40 [00:00<?, ?it/s]

epoch 10 test acc 0.8944754464285714


## 데이터 모델 저장하기

In [24]:
PATH = './'

torch.save(model, PATH + 'KoBERT_담화.pt')
torch.save(model.state_dict(), PATH + 'model_state_dict.pt')
torch.save({
    'model' : model.state_dict(),
    'optimizer' : optimizer.state_dict()
}, PATH + 'all.tar')

# 예측 모델

In [25]:
import pandas as pd

# 위에서 설정한 tok, max_len, batch_size, device를 그대로 입력
# comment : 예측하고자 하는 텍스트 데이터 리스트
def getSentimentValue(comment, tok, max_len, batch_size, device):
  commnetslist = [] # 텍스트 데이터를 담을 리스트
  emo_list = [] # 감성 값을 담을 리스트
  for c in comment: # 모든 댓글
    commnetslist.append( [c, 5] ) # [댓글, 임의의 양의 정수값] 설정
    
  pdData = pd.DataFrame( commnetslist, columns = [['댓글', '감성']] )
  pdData = pdData.values
  test_set = BERTDataset(pdData, 0, 1, tok, max_len, True, False) 
  test_input = torch.utils.data.DataLoader(test_set, batch_size=batch_size, num_workers=5)
  
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_input):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length 
    # 이때, out이 예측 결과 리스트
    out = model(token_ids, valid_length, segment_ids)
	
    # e는 2가지 실수 값으로 구성된 리스트
    # 0번 인덱스가 더 크면 부정, 긍정은 반대
    for e in out:
      if e[0]>e[1]: # 부정
        value = 0
      else: #긍정
        value = 1
      emo_list.append(value)

  return emo_list # 텍스트 데이터에 1대1 매칭되는 감성값 리스트 반환

In [26]:
data_train[0][1]

array(180, dtype=int32)

In [27]:
dataset_train[0][0]

'155,155,아파트 외벽 타고 올라가던 30대 외국인 떨어져 숨져 아파트에서 30대 외국인이 떨어져 숨져 <b>경찰</b>이 <b>수사</b>에 나섰다.....<b>경찰</b>에.. <b>경찰</b>은 목격자 진술을 통해 A씨가 아파트 외벽을 타고 올라가던 중 7층 에어컨 실외기에서 쉬다가 추락한 것으로 파악했다... <b>경찰</b>은 주변 인물들을 상대로 정확한 <b>사고</b> 경위를 조사하는 한편 부검을 통해 사인을 밝힐 계획이다...'

In [28]:
getSentimentValue([dataset_train[0][0],
                  dataset_train[1][0],
                  dataset_train[2][0],
                  dataset_train[3][0],
                  dataset_train[4][0],
                  dataset_train[5][0],
                  dataset_train[6][0],],tok,max_len,batch_size,device)

[0, 0, 0, 0, 0, 0, 0]

In [29]:
print(dataset_train[0][0],'\n',
     dataset_train[1][0],'\n',
     dataset_train[2][0],'\n',
     dataset_train[3][0],'\n',
     dataset_train[4][0],'\n',
     dataset_train[5][0],'\n',
     dataset_train[6][0],)

155,155,아파트 외벽 타고 올라가던 30대 외국인 떨어져 숨져 아파트에서 30대 외국인이 떨어져 숨져 <b>경찰</b>이 <b>수사</b>에 나섰다.....<b>경찰</b>에.. <b>경찰</b>은 목격자 진술을 통해 A씨가 아파트 외벽을 타고 올라가던 중 7층 에어컨 실외기에서 쉬다가 추락한 것으로 파악했다... <b>경찰</b>은 주변 인물들을 상대로 정확한 <b>사고</b> 경위를 조사하는 한편 부검을 통해 사인을 밝힐 계획이다... 
 "8540,8540,""전동 킥보드 법안 시간 소요… 인천시, 먼저 대책 마련 나선다 인천시가 최근 안전 논란이 불거진 전동 킥보드(중부일보 2020년 10월 6일 ㆍ11월 3일자 18면 보도) 등에 대한 대책 마련에 나섰다... .. .. ..또 이를 위해 안전정책과를 비롯해 교통정책과, 도로과, 인천<b>경찰</b>청, 인천시교육청, 군ㆍ구 등이 유기적으로 협력해야 한다고 강조했다...""" 
 "2670,2670,""에콰도르 <b>경찰</b>은 23일(현지 시각) 트위터에 과야킬과 엘투리, 코토팍시의 교도소 3곳에서 폭동으로 50명 이상의 재소자가 사망했다고 밝혔다...<b>범죄</b>조직이 이번 폭동 동기와 연관됐다는 추정이 나온다...“<b>범죄</b>조직이 전국 여러 교도소에서 동시에 폭력 행위를 벌였다”고 밝혔다.....<b>범죄</b>조직 조직원간의 다툼으로 11명이 숨지는 등 지난 한 해 동안 재소자 간..""" 
 4906,4906,남양주 비닐하우스 화재…불 끄려던 50대 화상 27일 오후 3시5분께 남양주시 진건읍의 한 농기구 보관용 비닐하우스에서 불이 나 약 25분 만에 꺼졌다.   인근에 있던 50대 A씨가 불을 끄려다 얼굴과 팔 등에 화상을 입어 병원으로 옮겨졌다.   <b>경찰</b>과 소방당국은 화재원인을 조사 중이다. 남양주=하지은기자  
 5590,5590,..<b>사고</b> 있다.....<b>수사</b>할 필요성이 있다고 보인다”며 게시자에게 정보 제공을 요구했다...“이 정도면 

In [30]:
getSentimentValue([dataset_train[0][0],
                  dataset_train[1][0],
                  dataset_train[2][0],
                  dataset_train[3][0],
                  dataset_train[4][0],
                  dataset_train[5][0],
                  dataset_train[6][0],
                  dataset_train[7][0],
                  dataset_train[8][0],
                  dataset_train[9][0],
                  dataset_train[10][0],
                  dataset_train[11][0],
                  dataset_train[12][0],
                  dataset_train[13][0],
                  dataset_train[14][0],],tok,max_len,batch_size,device)

[0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1]

In [32]:
import pickle
pickle.dump(model, open('model.pkl','wb'))